In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
!ls -ltr
!hostname -i

127.0.1.1 


# Loading UD modules

In [3]:
from __future__ import print_function
import torch
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from torchsummary import summary
from tqdm import tqdm
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import os 
import numpy as np


import model_utility.data_utils as dutils
import model_utility.model_utils as mutils
import model_utility.plot_utils as putils 
import model_utility.regularization as regularization
import model_file.model_cifar as model_cifar
import model_utility.alb_utils as alb

import model_file.models as mod

import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
import data_augmentation.albumentations as A
from data_augmentation.albumentations.pytorch import ToTensor

In [5]:
A.__version__
A.__file__
A.Compose.__module__
A.Cutout.__module__
A.RandomCrop.__module__
A.HorizontalFlip.__module__
A.ElasticTransform.__module__
A.CoarseDropout.__module__
A.Normalize.__module__
A.pytorch.ToTensor.__module__
A.pytorch.ToTensor.__module__

'0.4.6'

'/home/abhinav/Documents/EVA5/session9/data_augmentation/albumentations/__init__.py'

'data_augmentation.albumentations.core.composition'

'data_augmentation.albumentations.augmentations.transforms'

'data_augmentation.albumentations.augmentations.transforms'

...

'data_augmentation.albumentations.augmentations.transforms'

'data_augmentation.albumentations.augmentations.transforms'

'data_augmentation.albumentations.augmentations.transforms'

'data_augmentation.albumentations.pytorch.transforms'

'data_augmentation.albumentations.pytorch.transforms'

# Check GPU avaiability

In [6]:
device = dutils.get_device()

CUDA Available? True
Device is cuda:0


In [7]:
cuda = torch.cuda.is_available()
print(cuda)

True


# CIFAR10 Basics

In [8]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

SEED = 1
batch_size=128

# CUDA?
if device.type == 'cuda':
    torch.cuda.manual_seed(SEED)
else:
    torch.manual_seed(SEED)

# Load CIFAR10 dataset

In [9]:
os.getcwd()

'/home/abhinav/Documents/EVA5/session9'

In [10]:
path = os.getcwd()
path

'/home/abhinav/Documents/EVA5/session9'

In [11]:
batch_size = 128

In [12]:
# load CIFAR10 dataset
trainset, testset, train_loader, test_loader = alb.get_dataloader(batch_size=batch_size, num_workers=4, cuda = True,path = path)

Running over Cuda !!  True


KeyboardInterrupt: 

# Data mage Visualization

In [ ]:
#Plot 20 random images
putils.plot_dataset_images(device, classes, train_loader)

### Model stracture

In [ ]:
model = mod.ResNet18()

In [ ]:
mutils.model_summary(model, device, input_size=(3, 32, 32))

In [ ]:
# Create CIFAR10 network
# model = model_cifar.Net(dropout_value = 0.05)
# mutils.model_summary(model, device, input_size=(3, 32, 32))

# Build and train model

In [ ]:
from tqdm import tqdm

In [ ]:
# build and train the model
model_name = "Model CIFAR10"
EPOCHS = 50
model =  model.to(device) # use GPU

# using L1 and L2 regularization
train_acc, train_losses, test_acc, test_losses = mutils.build_model(model, 
                                                            device, 
                                                            trainloader=train_loader, 
                                                            testloader=test_loader, 
                                                            epochs=EPOCHS,
                                                            L1_loss_flag=False,
                                                            L2_penalty_val=0.001)

In [ ]:
A.__file__

# Test Accuracy

In [ ]:
mutils.get_test_accuracy(model, device, test_loader)

### Class level Accuracy

In [ ]:

mutils.class_based_accuracy(model, device, classes, test_loader)

# Visualization

In [ ]:
print("\nPloting Accuracy and Loss history: {}\n".format(model_name))
putils.plot_model_history(model_name, 
                   train_acc,test_acc,
                   train_losses,test_losses,
                   save_filename = "model_history")

# Misclassified images

In [ ]:
print ("\nPloting misclassified images: {}\n".format(model_name))

putils.miss_classification('Test',model, 
                          device, 
                          classes,
                          testloader=test_loader,
                          num_of_images = 20,
                          save_filename="model_misclassified")

In [ ]:
# from gradcam.grad_cam_viz import gradcam_plot
layer = 'layer4'
gradcam_plot(layer,model,testset,classes)